# ComputerAgent HUD Integration for OSWorld

This notebook demonstrates how to use the ComputerAgent with HUD for OSWorld benchmarking.
The ComputerAgent integration provides the same interface as OperatorAgent but works with both Claude and OpenAI models.

In [ ]:
# # Install dependencies if needed
# !uv venv 
# !source .venv/bin/activate
# !uv sync

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Required environment variables:
# - HUD_API_KEY (for HUD access)
# - ANTHROPIC_API_KEY (for Claude models)
# - OPENAI_API_KEY (for OpenAI models)

from pprint import pprint

## Quick single-task smoke test on OSWorld-Verified

The ComputerAgent integration can use Claude, OpenAI, UI-TARS, or composed models just like the original ComputerAgent:

In [1]:
from agent.integrations.hud import run_single_task

# Quick single-task smoke test on OSWorld-Verified-XLang
# You can swap "hud-evals/OSWorld-Verified-XLang" -> "hud-evals/SheetBench-V2" to test SheetBench.
await run_single_task(
    dataset="hud-evals/OSWorld-Verified-XLang",
    model="openai/computer-use-preview+openai/gpt-5-nano",  # or any supported model string
    task_id=155 # open last tab task (easy)
)

c:\Users\dillo\miniconda3\envs\cua\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



╔═════════════════════════════════════════════════════════════════╗
║                    🚀 See your agent live at:                   ║
╟─────────────────────────────────────────────────────────────────╢
║  https://app.hud.so/trace/cbe6f71b-f520-4630-9f27-778647070327  ║
╚═════════════════════════════════════════════════════════════════╝

Running: Can you make my computer bring back the last tab I shut down?


2025-08-27 16:17:53,047 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-08-27 16:18:02,697 - agent.ComputerAgent - INFO - LLM processing started with 5 messages
2025-08-27 16:18:15,887 - agent.ComputerAgent - INFO - LLM processing started with 7 messages
2025-08-27 16:18:28,541 - agent.ComputerAgent - INFO - LLM processing started with 9 messages
2025-08-27 16:18:42,176 - agent.ComputerAgent - INFO - LLM processing started with 11 messages
2025-08-27 16:18:55,937 - agent.ComputerAgent - INFO - LLM processing started with 13 messages
2025-08-27 16:19:11,654 - agent.ComputerAgent - INFO - LLM processing started with 15 messages
2025-08-27 16:19:23,839 - agent.ComputerAgent - INFO - LLM processing started with 17 messages
2025-08-27 16:19:39,065 - agent.ComputerAgent - INFO - LLM processing started with 19 messages
Tool execution failed: Tool evaluate has an output schema but did not return structured content
Evaluation phase failed: [MCPToolResult(meta=None, co

✅ Reward: 0.0

✓ Trace complete! View at: https://app.hud.so/trace/cbe6f71b-f520-4630-9f27-778647070327



## Run OSWorld-Verified in parallel

In [ ]:
import uuid
from agent.integrations.hud import run_full_dataset

# Full dataset evaluation (runs via HUD's run_dataset under the hood)
job_name = f"osworld-test-{str(uuid.uuid4())[:4]}"

results = await run_full_dataset(
    dataset="hud-evals/OSWorld-Verified-XLang",          # You can also pass a Dataset or a list[dict]
    job_name=job_name,                   # Optional; defaults to a timestamp for custom datasets
    model="openai/computer-use-preview", # Or any supported model string
    max_concurrent=20,                   # Tune to your infra
    max_steps=50,                        # Safety cap per task
    split="train[:3]"                    # Limit to just 3 tasks
)

# results is a list from hud.datasets.run_dataset; inspect/aggregate as needed
print(f"Job: {job_name}")
print(f"Total results: {len(results)}")
pprint(results[:3])  # preview

# Benchmark Composed Agents

In [ ]:
import uuid
from agent.integrations.hud import run_full_dataset

models_to_test = [
    "openai/computer-use-preview+anthropic/claude-opus-4-1-20250805",
    "anthropic/claude-opus-4-1-20250805+openai/computer-use-preview",
]


for model in models_to_test:
    # Full dataset evaluation (runs via HUD's run_dataset under the hood)
    job_uuid = str(uuid.uuid4())[:6]
    job_name = f"osworld {job_uuid} {model}"

    results = await run_full_dataset(
        dataset="hud-evals/OSWorld-Verified-XLang",
        job_name=job_name,                 
        model=model,
        max_concurrent=20,                   
        max_steps=75,
        trajectory_dir=f"trajectories/osworld_{job_uuid}"
    )

    # results is a list from hud.datasets.run_dataset; inspect/aggregate as needed
    print(f"Job: {job_name}")
    print(f"Total results: {len(results)}")
    pprint(results[:3])  # preview